In [5]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [6]:
import os
print(os.listdir())

['boot.py', 'hangspotmain.py', 'OLED_driver.py', 'BlueFly_funcs.py', 'lib', 'SI7021_funcs.py', 'OLED_grapher.py', 'BNO055_funcs.py', 'NewDataLogger_funcs.py', 'asyn.py', 'device_detection.py', 'TSL561_funcs.py', 'VL53L0X_funcs.py', 'VL6180_funcs.py', 'SDOF_funcs.py', 'SHT31D_funcs.py', 'BME280_funcs.py', 'MLX90621_funcs.py', 'BNO055serial_funcs.py', 'webserve_funcs.py', 'syslog.py', 'ds3231.py', 'ms5611.py', 'connectwifi.py', 'connecthotspot.py', 'lsq_funcs.py', 'syslog.txt', 'wificodes.txt', 'websocket_funcs.py', 'mqtt_as.py', 'config.py', 'thing.py', 'aswitch.py', 'h.txt', 'scriptlet.py', 'thing.txt', 'main.py', 'sdcard_funcs.py', 'UDPsocket_funcs.py', 'Dmain.py']


In [70]:
# Lessons on strings of bytes, hex and bin notation
# splitting and joining
print(list(b"\xAE\x00"))
print()

[174, 0]


In [ ]:
# Then UART interface datasheet

In [7]:
import time
from machine import UART

uart2 = UART(2, baudrate=115200, rx=2, tx=18)
# BNO055 SCL -> 18, SDA -> 2
#  (put this slide from the datasheet!)

I (51212) uart: ALREADY NULL


In [ ]:
# Need to make screenshots of the manual as we work through it

In [22]:
# Read the software version
uart2.write(bytes((0xAA, 0x01, 0x04, 2)))
time.sleep_ms(20)
k = uart2.read()
for x in k:
    print(hex(x))
    
# need slide of writing command

0xbb
0x2
0x11
0x3


In [23]:
# put into op mode
uart2.write(bytes((0xAA, 0x00, 0x3D, 1, 0x0C)))
time.sleep_ms(20)
v = uart2.read()
print(v)


b'\xee\x01'


In [24]:
# The 0x80 android notation
uart2.write(bytes((0xAA, 0x01, 0x3B, 1)))
time.sleep_ms(20)
print(uart2.read())


b'\xbb\x01\x80'


In [25]:
uart2.write(bytes((0xAA, 0x01, 0x1A, 6)))
time.sleep_ms(10)
v = uart2.read()
print(list(v))
# read the UNIT_SEL register

[187, 6, 75, 5, 155, 1, 159, 9]


In [27]:
import ustruct
# Now get the pitch roll and heading
uart2.write(bytes((0xAA, 0x01, 0x1A, 6)))
time.sleep_ms(10)
v = uart2.read()
print(v)
v = v[2:]
print((v[0]+v[1]*256)/16)
print([x/16  for x in ustruct.unpack("<hhh", v)])

b'\xbb\x06K\x05\x9b\x01\x9f\t'
84.6875
[84.6875, 25.6875, 153.9375]


In [28]:
uart2.write(bytes((0xAA, 0x01, 0x20, 24)))
time.sleep_ms(10)
v = uart2.read()
print(v)


b'\xbb\x18\xbc\x13\xfd\xd6\x01\xd3\x00\x00\x12\x00\xf1\xff\xdd\xff\xa9\x01}\xfe\xe6\xfc\x184\x0f\x00'


In [29]:
print(ustruct.unpack("<hhhhhhhhhhbB", v[2:]))  
# See; it's about 9.8m/s^2

(5052, -10499, -11519, 0, 18, -15, -35, 425, -387, -794, 24, 52)


In [57]:
# UDP technology
%serialconnect



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f53c03722e8, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [45]:
# Demo of available Wifis
import network
si = network.WLAN(network.STA_IF) 
si.active(True)
k = si.scan()
for x in k:
    print(x)


(b'IFP_55d1', b'\x1c\x1e\xe3\xeeU\xd1', 11, -52, 0, False)
(b'WifiGuest', b'\xa8\xb1\xd4\xc4\xb9\x11', 6, -54, 0, False)
(b'eduroam', b'\xa8\xb1\xd4\xc4\xb9\x10', 6, -55, 5, False)
(b'Clickshare-Typhoon', b'($\xff\xc4\xb7\x7f', 1, -57, 3, False)
(b'Marty Setup 8321179', b'b\x01\x94~\xf8\x9b', 1, -58, 0, False)
(b'Marty Setup 8358601', b'b\x01\x94\x7f\x8a\xc9', 1, -65, 0, False)
(b'MY WIFI 4563', b'\x9cO\xcf0Ec', 4, -72, 3, False)
(b'WifiGuest', b'\xa8\xb1\xd4\xc4\xb4Q', 11, -83, 0, False)
(b'eduroam', b'\xf0%r=\x96\x00', 1, -84, 5, False)
(b'WifiGuest', b'\xf0%r=\x96\x01', 1, -85, 0, False)


In [58]:
# Now do UDP packets 
import network, socket, time

si = network.WLAN(network.AP_IF) 
si.active(True)         
print(si.ifconfig())


('192.168.4.1', '255.255.255.0', '192.168.4.1', '0.0.0.0')


In [59]:
espname = "ESP_%X%X%X" % tuple(si.config("mac")[-3:])
print("\nConnect to wifi {}".format(espname))
port = 9019

sockudp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sockudp.settimeout(0.1)
sockudp.bind(("192.168.4.1", port))



Connect to wifi ESP_8C1FB2


In [60]:
sockudp.settimeout(1000)
rmess, udpaddr = sockudp.recvfrom(100)
print(rmess, udpaddr)

b'HiThereAMRC' ('192.168.4.3', 37245)


In [61]:
for i in range(1000):
    sockudp.sendto(b"T%d"%i, udpaddr)
    time.sleep_ms(100)


................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
KeyboardInterrupt: 


In [155]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [63]:
%sendtofile main.py

import network, socket, time, ustruct
from machine import UART

si = network.WLAN(network.AP_IF) 
si.active(True)         
port = 9019
sockudp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sockudp.settimeout(10000)
sockudp.bind(("192.168.4.1", port))
rmess, udpaddr = sockudp.recvfrom(100)
print("UDP socket", udpaddr)

uart2 = UART(2, baudrate=115200, rx=2, tx=18)

uart2.write(bytes((0xAA, 0x00, 0x3D, 1, 0x0C)))
time.sleep_ms(20)
print(uart2.read())

sockudp.sendto(b"n,t,qw,qx,qy,qz,lx,ly,lz,gx,gy,gz,temp,calibstat\n", udpaddr)
n = 0
while True:
    t = time.ticks_ms()
    uart2.write(bytes((0xAA, 0x01, 0x20, 24)))
    time.sleep_ms(4)
    v = uart2.read()
    if v is not None and len(v) == 26:
        qw,qx,qy,qz, lx,ly,lz, gx,gy,gz, temp,calibstat = \
            ustruct.unpack("<hhhhhhhhhhbB", v[2:])
        s = (n, t, qw,qx,qy,qz, lx/100,ly/100,lz/100, gx/100,gy/100,gz/100, temp,calibstat)
        m = b"%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%d,%d\n" % s
        sockudp.sendto(m, udpaddr)
        n += 1
    time.sleep_ms(6)


Sent 33 lines (1057 bytes) to main.py.


In [ ]:
# deal with calibration!!!